In [1]:
# Set project root
import os
import sys

# Manually set the path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from src.geometry import (
    compute_distance_from_homography,
    derive_metric_homography,
    recover_all_poses_from_homography,
    select_best_solution,
)
from src.matching import template_match
from src.data import load_measurements_from_yaml
from src.utils import load_calibration_json
from src.rendering import WebVisualizer

## **Analysis**

In [3]:
# Load measurement data
data = load_measurements_from_yaml("../assets/measurements.yaml")

# Load camera calibration
K, dist_coeffs, image_size = load_calibration_json("../assets/camera_calibration.json")

scenes = data.get_all_scenes()

In [4]:
def analyze_scene(scene_id, scenes, K):
    """
    This function analyzes a scene by executing the following steps:

    1. Load the scene and the corresponding templates.
    2. For each template, perform template matching against the scene image.
    3. Compute the homography and recover the camera pose.
    4. Compute the distance from the center of each template to the camera.
    5. Return the results including the template ID, homography, camera pose, and distance.
    """
    # Load scene and templates
    scene = data.get_scene(scene_id)
    templates = [data.get_template(t_id) for t_id in scenes[scene_id]]

    # Iterate over templates
    results = {}
    for template in templates:
        scene_path = os.path.join(project_root, scene.path)
        template_path = os.path.join(project_root, template.path)

        # Compute pixel-pixel homography
        H, mask, t_shape, reprojection_error = template_match(template_path, scene_path, extract_method='SIFT', match_method='BF', plot=False)

        template_size_px = plt.imread(template_path).shape[:2]

        # Derive the metric-pixel homography
        H_metric = derive_metric_homography(
            H_px=H,
            template_size_px=template_size_px,
            template_size_metric=(template.height, template.width)
        )

        # Template center point
        template_center_mm = np.array([template.width / 2, template.height / 2])

        # Compute the distance from the camera to the template center
        distance_pred = compute_distance_from_homography(
            H_metric=H_metric,
            K=K,
            point_metric=template_center_mm
        )

        # True distance and error
        distance_true = scene.get_distance('Camera', template.id).distance
        error = np.abs(distance_pred - distance_true)
        error_percent = (error / distance_true) * 100

        results[template.id] = {
            'homography': H_metric.tolist(),
            'distance_pred': distance_pred,
            'distance_true': distance_true,
            'error': error,
            'error_percent': error_percent,
        }

    return results

In [5]:
# Analyze scene
scene_id = 'S12'
results = analyze_scene(scene_id, scenes, K)

In [6]:
# Load template metadata
metadata = {t_id: data.get_template(t_id) for t_id in scenes[scene_id]}
metadata.keys()

dict_keys(['T2', 'T3', 'T4'])

In [7]:
# Create the web visualizer
visualizer = WebVisualizer(
    metadata=metadata,
    results=results,
    K=K
)

visualizer.generate_html()

  3D WEB VISUALIZATION GENERATOR

[1/5] Processing Template Poses
     ----------------------------------------
     | Generated 3 template meshes
     ----------------------------------------

[2/5] Creating Camera Visualization
     ----------------------------------------
     | Generated 4 line groups (axes + frustum)
     ----------------------------------------

[3/5] Loading HTML Template
     ----------------------------------------
     | Template loaded successfully
     ----------------------------------------

[4/5] Substituting Data Placeholders
     ----------------------------------------
     | Data integration complete
     ----------------------------------------

[5/5] Saving Visualization
     ----------------------------------------
     | File saved successfully
     ----------------------------------------

  VISUALIZATION COMPLETE!
  >> Open 'visualization.html' in your browser
  >> to view the 3D scene
